In [14]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import json
import os



In [2]:
spark = SparkSession.builder \
    .appName("AzureLogProcessing") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/03/11 12:44:34 WARN Utils: Your hostname, WDSCBELT456 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/11 12:44:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 12:44:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [46]:
file_list = os.listdir("/home/obiwan/dbverse_git/pyspark_examples/data/Azure_IR")
print(file_list)

['pipelineruns.json', 'activityruns.json', 'triggerruns.json']


In [47]:
file_list = [f"/home/obiwan/dbverse_git/pyspark_examples/data/Azure_IR/{file}" for file in file_list ]
print(file_list)

['/home/obiwan/dbverse_git/pyspark_examples/data/Azure_IR/pipelineruns.json', '/home/obiwan/dbverse_git/pyspark_examples/data/Azure_IR/activityruns.json', '/home/obiwan/dbverse_git/pyspark_examples/data/Azure_IR/triggerruns.json']


In [58]:
from functools import reduce

# Initialize Spark session (if not already initialized)
# List of JSON file paths to read

# Step 1: Read all JSON files into Spark DataFrames
df_list = [spark.read.option("lines","true").json(file) for file in file_list]

# Step 2: Handle Schema Differences and Union All DataFrames
df_final = reduce(lambda x, y: x.unionByName(y, allowMissingColumns=True), df_list)

from pyspark.sql.functions import to_json, col

# Convert 'properties' to JSON string before converting to Pandas
df_final = df_final.withColumn("properties", to_json(col("properties")))


# Step 3: Show Final Merged DataFrame
#display(df_final)

In [59]:
df_final.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------------+-------+--------------------+-------------+--------+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------+------------+--------------------+----------+----+--------------------+--------------------+----------------------+-------------------+------------------+--------------------+------------+--------------------+---------+---------------------------+--------------------+--------------------+--------------+------------+---------+-----------+-----------+
|           EventName|                body|            category|       correlationId|                 end|   env_dt_spanId|      env_dt_traceId|            env_name|            env_time|env_ver|             groupId|        level|location|                name|       operationN

In [65]:
import pandas as pd
import json
from pyspark.sql.types import StructType, StructField, StringType

# Step 1: Convert PySpark DataFrame to Pandas
pdf = df_final.toPandas()

# Ensure 'properties' is valid JSON
pdf["properties"] = pdf["properties"].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

# Function to flatten JSON
def flatten_json(nested_json, prefix=""):
    out = {}
    if isinstance(nested_json, dict):
        for key, value in nested_json.items():
            full_key = f"{prefix}_{key}" if prefix else key
            if isinstance(value, dict):
                out.update(flatten_json(value, full_key))
            elif isinstance(value, list):
                out[full_key] = json.dumps(value) if value else None  # Store lists as JSON
            else:
                out[full_key] = value
    else:
        out[prefix] = nested_json
    return out

# Step 2: Flatten 'properties' column dynamically
pdf_flattened = pdf["properties"].apply(lambda x: flatten_json(x) if isinstance(x, dict) else {})

# Convert to Pandas DataFrame
pdf_expanded = pd.DataFrame(pdf_flattened.tolist())

# Merge with original DataFrame (excluding old 'properties' column)
pdf_final = pd.concat([pdf.drop(columns=["properties"]), pdf_expanded], axis=1)

# Ensure correct data types before converting to Spark
for col in pdf_final.columns:
    if pdf_final[col].dtype == 'object':
        pdf_final[col] = pdf_final[col].astype(str)

# Explicit schema definition
schema = StructType([StructField(col, StringType(), True) for col in pdf_final.columns])

# Convert the Pandas DataFrame back to PySpark
df_spark_final = spark.createDataFrame(pdf_final, schema=schema)

# Show result in Spark
df_spark_final.show(truncate=False)


+----------------------+---------------------------------------------------------------------------------------------------------+-----------------------+------------------------------------+----------------------------+----------------+--------------------------------+-----------------------------------+----------------------------+-------+------------------------------------+-------------+--------+----------------------+--------------------------------+-------------+---------------------------------------------------------------------------------------------------------------------------------+------------------------------------+--------------+------------+----------------------------+----------+----+----------------------------+----------------------------+----------------------+-------------------+------------------+------------------------------------+------------+---------------------------------------------------------------------------------------------------------------------

In [62]:
df_spark_final.printSchema

<bound method DataFrame.printSchema of DataFrame[EventName: string, body: string, category: string, correlationId: string, end: string, env_dt_spanId: string, env_dt_traceId: string, env_name: string, env_time: string, env_ver: string, groupId: string, level: string, location: string, name: string, operationName: string, pipelineName: string, resourceId: string, runId: string, severityNumber: string, severityText: string, start: string, status: string, tags: string, time: string, timestamp: string, activityIterationCount: string, activityName: string, activityRetryCount: string, activityRunId: string, activityType: string, billingResourceId: string, cpuCoreMs: string, effectiveIntegrationRuntime: string, meterId: string, pipelineRunId: string, recoveryStatus: string, triggerEvent: string, triggerId: string, triggerName: string, triggerType: string, Annotations: string, Predecessors: string, SystemParameters_PipelineRunRequestTime: string, Message: string, SystemParameters_ExecutionStar

In [64]:
df_spark_final.repartition(1)\
.write.csv("/home/obiwan/dbverse_git/pyspark_examples/src/random_pocs/temp",mode="overwrite")